In [0]:
diamonds_df = spark.read.format("csv").option("header", "true").load("/databricks-datasets/Rdatasets/data-001/csv/ggplot2/diamonds.csv")


In [0]:
diamonds_df.write.format("delta").mode("overwrite").saveAsTable("diamond")


In [0]:
diamonds_df.count()

Out[3]: 53940

In [0]:
diamonds_df.show()

+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|_c0|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|  1| 0.23|    Ideal|    E|    SI2| 61.5|   55|  326|3.95|3.98|2.43|
|  2| 0.21|  Premium|    E|    SI1| 59.8|   61|  326|3.89|3.84|2.31|
|  3| 0.23|     Good|    E|    VS1| 56.9|   65|  327|4.05|4.07|2.31|
|  4| 0.29|  Premium|    I|    VS2| 62.4|   58|  334| 4.2|4.23|2.63|
|  5| 0.31|     Good|    J|    SI2| 63.3|   58|  335|4.34|4.35|2.75|
|  6| 0.24|Very Good|    J|   VVS2| 62.8|   57|  336|3.94|3.96|2.48|
|  7| 0.24|Very Good|    I|   VVS1| 62.3|   57|  336|3.95|3.98|2.47|
|  8| 0.26|Very Good|    H|    SI1| 61.9|   55|  337|4.07|4.11|2.53|
|  9| 0.22|     Fair|    E|    VS2| 65.1|   61|  337|3.87|3.78|2.49|
| 10| 0.23|Very Good|    H|    VS1| 59.4|   61|  338|   4|4.05|2.39|
| 11|  0.3|     Good|    J|    SI1|   64|   55|  339|4.25|4.28|2.73|
| 12| 0.23|    Ideal|    J|    VS1

In [0]:
spark.sql("select count(*) from diamond").show()

+--------+
|count(1)|
+--------+
|   53940|
+--------+



In [0]:
%sql
select count(*) from diamond


count(1)
53940


In [0]:
%sql

DESCRIBE DETAIL diamond

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,9fc8ee5a-ffc7-41a9-89fe-3d636d7cc296,spark_catalog.default.diamond,null,dbfs:/user/hive/warehouse/diamond,2023-02-11T14:10:34.269+0000,2023-02-11T14:10:50.000+0000,List(),1,755428,Map(),1,2


In [0]:
%fs

ls /user/hive/warehouse/diamond

path,name,size,modificationTime
dbfs:/user/hive/warehouse/diamond/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/diamond/part-00000-c28cd93e-f5be-46e9-a167-c72a409e5b00-c000.snappy.parquet,part-00000-c28cd93e-f5be-46e9-a167-c72a409e5b00-c000.snappy.parquet,755428,1676124644000


In [0]:
%sql

DESCRIBE TABLE diamond

col_name,data_type,comment
_c0,string,null
carat,string,null
cut,string,null
color,string,null
clarity,string,null
depth,string,null
table,string,null
price,string,null
x,string,null
y,string,null


In [0]:
from pyspark.sql.functions import avg

filtered_df = diamonds_df.filter((diamonds_df["cut"] == "Ideal") & (diamonds_df["price"] > 1000))
grouped_df = filtered_df.groupBy("clarity").agg(avg("price"))
ordered_df = grouped_df.orderBy(grouped_df["avg(price)"].desc())

In [0]:
ordered_df.show()

+-------+------------------+
|clarity|        avg(price)|
+-------+------------------+
|    SI2| 5457.046354635463|
|    SI1| 4890.193991690636|
|    VS1| 4820.202886597938|
|    VS2| 4744.693523396343|
|     I1| 4642.622222222222|
|   VVS2| 4598.633727810651|
|   VVS1|   3575.6672091131|
|     IF|3532.8973561430794|
+-------+------------------+



In [0]:
%sql
SELECT clarity, AVG(price) as avg_price
FROM (
  SELECT *
  FROM diamond
  WHERE cut = 'Ideal' AND price > 1000
)
GROUP BY clarity
ORDER BY avg_price DESC

clarity,avg_price
SI2,5457.046354635463
SI1,4890.193991690636
VS1,4820.202886597938
VS2,4744.693523396343
I1,4642.622222222222
VVS2,4598.633727810651
VVS1,3575.6672091131
IF,3532.8973561430794
